# Chapter 4: Training Models

## 4.1 Linear Regression

### 4.1.1 The Normal Equation

### 4.1.2 Computational Complexity

## 4.2 Gradient Descent

### 4.2.1 Batch Gradient Descent

### 4.2.2 Stochastic Gradient Descent

### 4.2.3 Mini-batch Gradient Descent

## 4.3 Polynomial Regression

## 4.4 Learning Curves

## 4.5 Regularized Linear Models

### 4.5.1 Ridge Regression

### 4.5.2 Lasso Regression

### 4.5.3 Elastic Net

### 4.5.4 Early Stopping

## 4.6 Logistic Regression

### 4.6.1 Estimating Probabilities

### 4.6.2 Training and Cost Function

### 4.6.3 Decision Boundaries

### 4.6.4 Softmax Regression